라이브러리 설치

In [ ]:
# !pip install langchain-teddynote arrow vllm langchain-community rank_bm25 qdrant_client langchain-qdrant guardrails-ai torch ragas faiss-cpu

# RAGAS 평가 준비
---

In [2]:
# LangSmith 추적 설정 (응답 결과 저장을 위함)
import os
from langchain_teddynote import logging

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "YOUR_API_KEY"
os.environ["LANGSMITH_PROJECT"] = "LANGCHAIN_PROJECT"

logging.langsmith("LANGCHAIN_PROJECT")

LangSmith 추적을 시작합니다.
[프로젝트명]
LANGCHAIN_PROJECT


In [3]:
# 테스트 데이터셋 확인
import pandas as pd

df = pd.read_csv("notice.me")
print(df.ground_truth[104])

안녕하세요! 🌿
1. 8월 1일 진행되는 파이널 프로젝트와 수료식 참석 여부 조사가 진행 중이며, 마지막 교육 일정의 도시락 제공을 위해 7월 25일(금) 오후 1시까지 구글폼으로 참여 신청을 해주세요.
2. 파이널 프로젝트 예선전 발표 자료는 계획서·결과 보고서·시연영상을 포함하여 7월 30일(수) 오후 5시까지 구글폼으로 제출해야 하며, 본선 자료는 여기에 프로젝트 완료 보고서를 추가하여 구글 드라이브 폴더에 제출해야 합니다.
3. 제출 폴더와 필수 양식 드라이브 링크가 제공되며, 작성 중 궁금한 사항은 운영팀이나 사무실에 문의할 수 있습니다.
4. 6단위기간 종료에 따른 출석률 메일이 발송되며, 이의 신청은 7월 25일(금) 오후 3시까지 DM으로 할 수 있습니다. 훈련 장려금 신청부터 지급까지는 약 한 달이 걸리며, 계좌 정보는 고용24에서 확인하고 문의는 8월 25일 이후 가능합니다.
5. 예선전 발표 순서 추첨은 7월 30일(수) 오전 10시에 타운홀에서 진행되고 발표 시간은 팀당 17분입니다. 예선전은 7월 31일(목) 오전 9시부터 진행되며, 상위 5팀이 본선에 진출하고 우수 발표팀에게는 특별상이 주어집니다.
🙏 모두 파이널 프로젝트와 수료식 준비 잘하시어 좋은 결과 얻으시길 바랍니다.


In [4]:
from datasets import Dataset
import pandas as pd

# 기존 DataFrame(df)을 HuggingFace Dataset으로 단순 변환
test_dataset = Dataset.from_pandas(df)
print(test_dataset)

# RAGAS 평가를 위해 새로운 DataFrame 생성
df_for_ragas = pd.DataFrame({
    "question": df["question"],                # 기존 질문
    "answer": df["ground_truth"],              # LLM 응답 대신 Ground Truth 사용 가능
    "contexts": df["context"].apply(lambda x: [x]),  # context를 리스트로 변환
    "ground_truth": df["ground_truth"]         # 평가 기준 정답 유지
})

# HuggingFace Dataset으로 다시 변환
# 이제 RAGAS에서 바로 사용 가능한 형태로 준비 완료
test_dataset = Dataset.from_pandas(df_for_ragas)

# 변환된 Dataset의 첫 번째 샘플 확인
print(test_dataset[0])


Dataset({
    features: ['question', 'context', 'ground_truth', 'evolution_type', 'meta', 'domain', 'difficulty', 'language', 'episode_done'],
    num_rows: 110
})
{'question': '자격증 응시료는 어떻게 환급받나요?', 'answer': '안녕하세요! 🌿\n1. 교육 과정과 관련된 자격증 취득 시 응시료 전액을 수료 후 1개월 내 환급받을 수 있으며, 지원 대상은 해당 과정 수료자입니다.\n2. 합격증과 매출전표를 제출해야 하며, 민간 자격증은 지원되지 않습니다.\n3. 관련 내용은 문서에서 확인해주시기 바랍니다.\n🙏 추가로 궁금한 사항이 있으면 언제든 문의해주세요', 'contexts': ['교육 과정과 관련된 자격증 취득 시 응시료 전액을 수료 후 1개월 내 환급받을 수 있으며, 지원 대상은 해당 과정 수료자입니다. 합격증과 매출전표를 제출해야 하며, 민간 자격증은 지원되지 않습니다.'], 'ground_truth': '안녕하세요! 🌿\n1. 교육 과정과 관련된 자격증 취득 시 응시료 전액을 수료 후 1개월 내 환급받을 수 있으며, 지원 대상은 해당 과정 수료자입니다.\n2. 합격증과 매출전표를 제출해야 하며, 민간 자격증은 지원되지 않습니다.\n3. 관련 내용은 문서에서 확인해주시기 바랍니다.\n🙏 추가로 궁금한 사항이 있으면 언제든 문의해주세요'}


# 여기서부터 서비스 실행란
---

app/models/embedding_model.py

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

app/core/document_loader.py

In [6]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.schema import Document
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import re
from datetime import datetime
from langchain_qdrant import QdrantVectorStore
from qdrant_client.models import PayloadSchemaType, PointStruct

loader = TextLoader("/content/notice.md")
docs = loader.load()

import re
from datetime import datetime
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.schema import Document

# 날짜 추출 함수
def extract_date_from_markdown(text: str) -> str | None:
    match = re.search(
        r"##\s*Date\s*[:：]?\s*(\d{4})[.\-](\d{2})[.\-](\d{2})",
        text,
        re.IGNORECASE,
    )
    if match:
        try:
            return datetime.strptime("-".join(match.groups()), "%Y-%m-%d").date().isoformat()
        except ValueError:
            return None
    return None

# Markdown 문서들을 헤더 기준으로 분할
def load_and_split_markdown_docs() -> list[Document]:
    loader = TextLoader("/content/notice.md")
    docs = loader.load()

    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[("#", "Header 1")],
        strip_headers=False
    )

    all_splitted = []
    for doc in docs:
        current_date = None
        current_date_int = None
        header_splitted = splitter.split_text(doc.page_content)

        for s in header_splitted:
            maybe_new_date = extract_date_from_markdown(s.page_content)
            if maybe_new_date:
                current_date = maybe_new_date
                try:
                    current_date_int = int(current_date.replace("-", ""))
                except ValueError:
                    current_date_int = 0

            s.metadata.update(doc.metadata)
            s.metadata["date"] = current_date or ""
            s.metadata["date_int"] = current_date_int or 0

        all_splitted.extend(header_splitted)

    return all_splitted

app/core/recent_docs_cache.py

In [7]:
from functools import lru_cache
from langchain.schema import Document
# from app.core.document_loader import load_and_split_markdown_docs

@lru_cache(maxsize=1)
def get_top_recent_docs(k: int = 3) -> list[Document]:
    docs = load_and_split_markdown_docs()
    sorted_docs = sorted(docs, key=lambda d: d.metadata.get("date_int", 0), reverse=True)
    return sorted_docs[:k]

scripts/create_vectorstore.py

In [8]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
# from app.models.embedding_model import get_embedder
from langchain_text_splitters import MarkdownHeaderTextSplitter
from datetime import datetime
import re
import os

# 1. Markdown 문서 로드 (docs/ 폴더 내 .md 파일 대상)
loader = TextLoader("notion_general.md")
docs = loader.load()


# 2. 날짜 추출 함수
def extract_date_from_markdown(text: str) -> str | None:
    match = re.search(r"^##\s+(\d{4}-\d{2}-\d{2})", text, re.MULTILINE)
    if match:
        try:
            date_obj = datetime.strptime(match.group(1), "%Y-%m-%d").date()
            return date_obj.isoformat()
        except ValueError:
            return None
    return None

# 3. Markdown 문서들을 헤더 기준으로 분할
def split_docs_by_markdown_headers(docs: list[Document]) -> list[Document]:
    headers_to_split_on = [("#", "Header 1")]
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False
    )

    all_splitted = []
    for doc in docs:
        header_splitted = splitter.split_text(doc.page_content)
        # 메타데이터
        for s in header_splitted:
            # 문서 블록에서 날짜(예: ## 2025-06-01)를 추출하여 메타데이터로 저장
            date = extract_date_from_markdown(s.page_content)
            s.metadata.update(doc.metadata)
            # "최근" 같은 사용자 질문을 처리할 수 있도록 날짜 정보를 메타데이터에 포함
            s.metadata["date"] = date or ""
        all_splitted.extend(header_splitted)

    return all_splitted

header_splitted_docs = split_docs_by_markdown_headers(docs)

# 4. 문서 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
)
chunks = splitter.split_documents(header_splitted_docs)

# 5. 벡터스토어 생성
vectorstore = FAISS.from_documents(chunks, embedding=embedding)

llm_client.py

In [9]:
# from app.models.qwen3_loader import tokenizer, llm, sampling_params
from langsmith import traceable
import re
# from dotenv import load_dotenv

# load_dotenv()

# 시스템 메시지
SYSTEM_MESSAGE = "You are a kind and friendly chatbot for announcements who responds based on the previous conversation flow. Always answer in Korean."

# 공통 메시지 생성 및 프롬프트 구성 함수
def build_prompt(user_input: str, context: str = "") -> str:
    full_content = f"{context}\n\n{user_input}" if context else user_input

    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": full_content}
    ]

    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
        )

# 비동기 generator(agen)에서 마지막 응답 요소만 수집
async def get_last_output(agen) -> str:
    last_text = ""
    async for item in agen:
        if item.outputs and item.outputs[0].text:
            last_text = item.outputs[0].text
    return last_text

# 공통 LLM 호출 유틸 함수
async def llm_generate(prompt_str: str, request_id: str) -> str:
    agen = llm.generate(prompt_str, sampling_params, request_id=request_id)
    result = await get_last_output(agen)
    return result if result else "empty:" + prompt_str

# 요약/뉴스 생성 전용 함수
@traceable(name="Summary LLM Response", inputs={"프롬프트": lambda args, kwargs: args[0]})
async def get_summarize_response(prompt: str, request_id: str) -> str:
    prompt_str = build_prompt(prompt)
    return await llm_generate(prompt_str, request_id)

# 문서 기반 챗봇 응답 함수
@traceable(name="Chat LLM Response", inputs={"질문": lambda args, kwargs: args[0]})
async def get_chat_response(prompt: str, docs: str, request_id: str) -> str:
    prompt_str = build_prompt(prompt)
    return await llm_generate(prompt_str, request_id)

# 스트리밍 기반 챗봇 응답 함수
def get_chat_response_stream(prompt: str, docs, request_id: str):
    sent_text = ""
    prompt_str = build_prompt(prompt)
    agen = llm.generate(prompt_str, sampling_params, request_id=request_id)

    async def _wrapper():
        nonlocal sent_text
        async for result in agen:
            text = result.outputs[0].text
            new_text = text[len(sent_text):]
            sent_text = text

            for word in re.findall(r'\s+|\S+', new_text):
                yield word

            if result.finished:
                return

    return _wrapper()


app/models/qwen3_loader.py

In [ ]:
from vllm import SamplingParams, AsyncEngineArgs, AsyncLLMEngine
from transformers import AutoTokenizer

# 토크나이저 로딩 (프롬프트용 메시지 생성에 사용)
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-14B-AWQ",
    trust_remote_code=True
)

engine_args = AsyncEngineArgs(
    model="Qwen/Qwen3-14B-AWQ",
    gpu_memory_utilization=0.8,
    tensor_parallel_size=1
)

llm = AsyncLLMEngine.from_engine_args(engine_args)

sampling_params = SamplingParams(
    temperature=0.6,           # 답변 다양성 (우린 정보형 챗봇이라 높을 필요없음)
    top_p=0.95,                # 핵심 단어 생성 기준
    max_tokens=1024,           # 제한 토큰 수
    stop=["</s>"]      # 이 토큰이 생성되면 답변 중단
)

app/core/chat_history.py

In [11]:
# 챗봇 히스토리 저장 (인메모리)
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from pydantic import BaseModel, Field
from typing import List

MAX_HISTORY_MESSAGES = 6  # 최대 저장 메시지 수

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)

    def _trim(self):
        """최신 MAX_HISTORY_MESSAGES 개만 유지"""
        if len(self.messages) > MAX_HISTORY_MESSAGES:
            self.messages = self.messages[-MAX_HISTORY_MESSAGES:]

    def add_messages(self, messages: list[BaseMessage]) -> None:
        self.messages.extend(messages)
        self._trim()

    def clear(self) -> None:
        self.messages = []

    def get_messages(self) -> list[BaseMessage]:
        return self.messages

    def add_user_message(self, message: str) -> None:
        self.messages.append(HumanMessage(content=message))
        self._trim()

    def add_ai_message(self, message: str) -> None:
        self.messages.append(AIMessage(content=message))
        self._trim()

# 사용자별 메모리
store = {}
def get_session_history(user_id: str) -> BaseChatMessageHistory:
    if (user_id) not in store:
        store[(user_id)] = InMemoryHistory()
    return store[(user_id)]

# 히스토리를 문자열로 반환
def chat_history_to_string(history: InMemoryHistory) -> str:
    messages = history.get_messages()

    lines = []
    for msg in messages:
        role = "사용자" if msg.type == "human" else "AI"
        lines.append(f"{role}: {msg.content}")

    return "\n".join(lines)

app/core/date_utils.py

In [12]:
import arrow
import re
from holidays import Korea

# 공휴일 및 마감 키워드 설정
holidays = Korea(years=[2025])
DEADLINE_KEYWORDS = ["신청", "접수", "지원", "마감", "등록"]

# 마감 키워드 포함 여부 확인
def contains_deadline_keyword(text: str) -> bool:
    return any(kw in text for kw in DEADLINE_KEYWORDS)

# 평일 기준 N일 전 날짜 계산
def subtract_business_days(date: arrow.Arrow, days: int, holidays: set = set()) -> arrow.Arrow:
    count = 0
    while count < days:
        date = date.shift(days=-1)
        date_str = date.format('YYYY-MM-DD')
        if date.weekday() < 5 and date_str not in holidays:
            count += 1
    return date

# 상대 날짜 문자열 → 절대 날짜 변환 (마감일 추가)
def parse_relative_dates(text: str, tz: str = 'Asia/Seoul'):
    now = arrow.now(tz)
    weekday_kr = ["월", "화", "수", "목", "금", "토", "일"]
    add_deadline = contains_deadline_keyword(text)

    # 반환 모드 (향후 최신 문서 모드 처리)
    mode_info = {"mode": None}

    # 상대 날짜 패턴 매핑
    patterns = {
        r"오늘": now,
        r"어제": now.shift(days=-1),
        r"그제": now.shift(days=-2),
        r"내일": now.shift(days=+1),
        r"모레": now.shift(days=+2),
        r"이번\s*주\s*월요일": now.floor('week').shift(days=0),
        r"이번\s*주\s*화요일": now.floor('week').shift(days=1),
        r"이번\s*주\s*수요일": now.floor('week').shift(days=2),
        r"이번\s*주\s*목요일": now.floor('week').shift(days=3),
        r"이번\s*주\s*금요일": now.floor('week').shift(days=4),
        r"이번\s*주\s*토요일": now.floor('week').shift(days=5),
        r"이번\s*주\s*일요일": now.floor('week').shift(days=6),
        r"다음\s*주\s*월요일": now.floor('week').shift(weeks=+1, days=0),
        r"다음\s*주\s*화요일": now.floor('week').shift(weeks=+1, days=1),
        r"다음\s*주\s*수요일": now.floor('week').shift(weeks=+1, days=2),
        r"다음\s*주\s*목요일": now.floor('week').shift(weeks=+1, days=3),
        r"다음\s*주\s*금요일": now.floor('week').shift(weeks=+1, days=4),
        r"다음\s*주\s*토요일": now.floor('week').shift(weeks=+1, days=5),
        r"다음\s*주\s*일요일": now.floor('week').shift(weeks=+1, days=6),
    }

    # 상대 날짜 치환 (첫 번째 매칭만)
    for pattern, date in patterns.items():
        if re.search(pattern, text):
            target_date = date.format('YYYY-MM-DD')
            target_weekday = weekday_kr[date.weekday()]
            replacement = f"{target_date} ({target_weekday})"

            if add_deadline:
                deadline_date = subtract_business_days(date, 3, holidays)
                deadline_fmt = deadline_date.format('YYYY-MM-DD')
                deadline_weekday = weekday_kr[deadline_date.weekday()]
                replacement += f" (신청 마감: {deadline_fmt} ({deadline_weekday}))"

            text = re.sub(pattern, replacement, text)
            break

    # 특정 날짜(예: 6월 5일) → 절대 날짜 변환
    date_patterns = re.findall(r"(?:(\d{4})년\s*)?(\d{1,2})월\s*(\d{1,2})일", text)
    for match in date_patterns:
        year = int(match[0]) if match[0] else now.year
        month = int(match[1])
        day = int(match[2])
        try:
            date = arrow.get(f"{year}-{month:02d}-{day:02d}")
            target = date.format('YYYY-MM-DD')
            weekday = weekday_kr[date.weekday()]
            replacement = f"{target} ({weekday})"

            if add_deadline:
                deadline = subtract_business_days(date, 3, holidays)
                deadline_fmt = deadline.format('YYYY-MM-DD')
                deadline_weekday = weekday_kr[deadline.weekday()]
                replacement += f" (신청 마감: {deadline_fmt} ({deadline_weekday}))"

            original = f"{match[0]+'년 ' if match[0] else ''}{month}월 {day}일"
            text = text.replace(original, replacement)
        except Exception:
            continue

    # 최신 관련 키워드 감지
    if re.search(r"최근|최신", text):
        mode_info["mode"] = "latest"

    return text, mode_info

app/models/prompt_template.py

In [13]:
chatbot_rag_prompt = """
사용자와의 이전 대화 기록, 출력형식, 예시, 출력조건에 맞추어 응답하세요.

[출력 형식]:
1. 인사말: "안녕하세요! 🌿"
2. 본문 안내: 항목별 번호로 정리 (가능하면 강조나 이모지도 적절히 포함)
3. 끝인사: "🙏 [격려 또는 주의사항 등]"

[예외 응답]:
- 공지사항과 무관한 질문: "카카오테크 부트캠프 관련 공지사항만 질문해주세요 😃"

----
[출력 형식 예시]:
예시 1
질문 : "다음주 화요일에 휴가 신청하는 법을 알려줘"
안녕하세요! 🌿
2025-05-20(화) 휴가 신청 방법에 대해 안내드립니다.
1. 휴가는 평일 기준 최소 3일 전, 👉 **2025-05-14(수)**까지 통합신청센터를 통해 미리 신청해 주세요.(※ 주말과 공휴일은 제외됩니다.)
2. 출석입력요청대장과 휴가신청서를 하나의 PDF로 묶어 제출해 주세요.
3. PDF 파일명은 [yy.mm.dd. 한글명 사유] 형식으로 작성해 주세요.
4. 휴가는 단위기간별로 1개씩, 총 6회까지 사용하실 수 있습니다.
🙏 신청 기한을 넘기면 휴가 사용이 제한될 수 있으니, 꼭 여유 있게 준비해 주세요!

---
[출력 조건]:
1. 문서와 주제(공지사항)와 명백히 무관한 경우에만 아래 문장만 출력합니다:
   '카카오테크 부트캠프 관련 공지사항만 질문해주세요 😃'
2. 문서와 관련된 질문에는 요청된 출력 형식에 따라 정확하게 응답합니다.
3. 모든 문장은 '~해주세요'와 같은 존댓말로 작성합니다.
4. 최근/최신 관련 사용자 질의에 대해 날짜 정보를 참고하여 응답해주세요.
5. 휴가일 기준 평일(영업일) 기준 3일 전에 신청해야 한다는 규칙을 반드시 적용해 정확한 마감일을 안내합니다.(예: 2025-05-20(화) 휴가일 → 2025-05-15(목)까지 신청해야 합니다.)

---
[History]:
{history}

[Context]:
{context}

[Question]:
{question}
"""


app/services/chat_service_v3.py

In [14]:
# from app.core.date_utils import parse_relative_dates
# from app.core.chat_history import get_session_history, chat_history_to_string
# from app.core.recent_docs_cache import get_top_recent_docs
# from app.core.retriever_factory import create_ensemble_retriever
# from app.models.prompt_template import chatbot_rag_prompt
# from app.models.llm_client import get_chat_response_stream
from langsmith.run_helpers import get_current_run_tree
from langsmith import traceable
from langchain.retrievers import BM25Retriever, EnsembleRetriever


# BM25 리트리버 생성 함수
def create_bm25_retriever(header_splitted_docs):
    # 텍스트 기반 검색기
    bm25_retriever = BM25Retriever.from_documents(
        header_splitted_docs,
        )
    bm25_retriever.k = 1
    return bm25_retriever

# FAISS 리트리버 생성 함수
def create_faiss_retriever(vectorstore):
    # 벡터 기반 검색기
    retriever = vectorstore.as_retriever(
        search_type="similarity_score_threshold",         # 유사도 점수 기반 필터링
        search_kwargs={"k": 2, "score_threshold": 0.7}
    )
    return retriever

# 앙상블 리트리버 생성 함수
def create_ensemble_retriever(header_splitted_docs, vectorstore, weights=(0.3, 0.7)):
    bm25 = create_bm25_retriever(header_splitted_docs)
    faiss = create_faiss_retriever(vectorstore)
    return EnsembleRetriever(retrievers=[bm25, faiss], weights=list(weights), k=3)


# 최종 리트리버 생성
retriever = create_ensemble_retriever(header_splitted_docs, vectorstore)

#------------------------------

# 사용자별 세션 히스토리에 Q/A message 저장
async def save_chat_history(userId: int, question: str, answer: str):
    history = get_session_history(userId)
    history.add_user_message(question)
    history.add_ai_message(answer)

#------------------------------

@traceable(name="Chat Service V3", inputs={"질문": lambda args, kwargs: args[0]})
async def ragas_chat_service(question: str, request_id: str, userId: int):

    # 질문 전처리 (상대 날짜 -> 절대 날짜)
    parsed_question, mode_info = parse_relative_dates(question)

    # 사용자 히스토리 로딩 및 문자열 반환
    history = get_session_history(userId)
    history_str = chat_history_to_string(history)

    # 리트리버를 통해 질문에 관련된 문서를 BM25 + FAISS 앙상블 방식으로 검색
    docs = retriever.get_relevant_documents(parsed_question)

    if not docs:
        return "카카오테크 부트캠프 관련 공지사항만 질문해주세요 😃"

    context = "\n\n".join([doc.page_content for doc in docs])

    # "최근" 또는 "최신" 키워드가 원래 질문에 포함되어 있으면 top_docs 추가
    top_docs = get_top_recent_docs(k=3)
    if "최근" in question or "최신" in question:
        context += "\n\n" + "\n\n".join([doc.page_content for doc in top_docs])

    # 프롬프트 정의 및 LLM 호출
    prompt = chatbot_rag_prompt.format(history=history_str, context=context, question=parsed_question)

    agen = get_chat_response_stream(prompt, docs, request_id)
    answer_collector = []
    async for chunk in agen:
        answer_collector.append(chunk)

    full_answer = "".join(answer_collector)
    await save_chat_history(userId, parsed_question, full_answer)

    return full_answer  # 최종 답변 반환

# RAGAS 평가 시작
---

In [ ]:
import asyncio
import pandas as pd
from datasets import Dataset
import uuid

# 결과 저장 리스트
answers = []

# 질문 순회하며 응답 생성
async def generate_answers():
    for i, row in df.iterrows():
        ans = await ragas_chat_service(
            question=row["question"],
            request_id=str(uuid.uuid4()),
            userId=123  # 사용자 ID는 테스트용 고정값
        )
        answers.append(ans)

# 비동기 실행
await generate_answers()

# DataFrame에 응답 컬럼 추가
df["answer"] = answers

# contexts 컬럼 생성 (RAGAS 요구사항: 리스트)
df["contexts"] = df["context"].apply(lambda x: [x])

# Hugging Face Dataset 변환
dataset = Dataset.from_pandas(df[["question", "answer", "contexts", "ground_truth"]])

print(dataset[0])

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

sample_dataset = test_dataset.select(range(20))

from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=sample_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,

    ],
)
result

In [19]:
print(result)

{'context_precision': 0.9500, 'faithfulness': 0.6121, 'answer_relevancy': 0.1592, 'context_recall': 0.4475}
